<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/Logo_INSAvilletoulouse-RVB.png" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# [Scénarios d'Exploration Statistique](https://github.com/wikistat/Exploration)

# Initiation à l'AFC avec <a href="https://cran.r-project.org/"><img src="https://cran.r-project.org/Rlogo.svg" style="max-width: 40px; display: inline" alt="R"/></a>
# de la SVD à l'AFC - exemple élémentaire
#### Résumé 
Décomposition en valeurs singulières [SVD](http://wikistat.fr/pdf/st-m-explo-alglin.pdf) d'une matrice rectangulaire comme introduction à l'[Analyse Factorielle des Correspondances](http://wikistat.fr/pdf/st-m-explo-afc.pdf) (AFC); illustration sur des données élémentaires avec R puis avec le package [`FactoMineR`](http://factominer.free.fr/).

#### Avertissement
Les différents travaux et analyses proposés tout au long de ces documents sont largement explicités. Les commandes en R ou Python toutes fournies. L'important n'est pas de trouver la bonne syntaxe des commandes ni de finir au plus vite mais de réfléchir sur les méthodes, leurs conditions d'applications, les résultats obtenus. L'apprentissage de ces logiciels et de leur programmation est dispensés dans d'autres tutoriels.

Répondre aux questions **Q** tout au long de la réalisation.

## 1 Données
Les données sont une table de contingence croisant une variable de catégorie socioprofessionnelle avec un type d'hébergement en vacances mais sous une forme particulière car archivée en trois colonnes. La première contient l'effectif conjoint des modalités contenus dans les colonnes suivantes. La deuxième colonne est le code de catégorie socioprofessionnelle et la 3ème  le code correspondant à un type d'hébergement.

### Lecture, description des données
Les donnéees sont dans le fichier `vaccsp.dat`. Attention , les données ne sont pas sous une forme adaptée à la fonction `table`; il faut utiliser la fonction `xtab` pour définir la table de contingence associée.

In [ ]:
vaccsp=read.table("Data/vaccsp.dat",col.names=c("effectif","CSP","heber"))
summary(vaccsp)

In [ ]:
# construire la table de contingence
T=xtabs(effectif~CSP+heber,vaccsp)

In [ ]:
# graphe associé
mosaicplot(T)

**Q** Quel est ce graphique? Est-il facile à interpréter?

In [ ]:
# test du chi2
chisq.test(T)

**Q** Conclusion sur la description de ces variables, de leur liaison.

## 2 AFC pas à pas
### 2.1 Calcul des marges, métriques et profils.

In [ ]:
# Fréquences marginales ou barycentres
n=sum(T); gc=apply(T,2,sum)/n; gr=apply(T,1,sum)/n

In [ ]:
# Métriques
Dr=diag(gr); Dc=diag(gc)
Dr1=diag(1/gr); Dc1=diag(1/gc)

In [ ]:
# Profils lignes et colonnes
A=t(T)%*%Dr1/n; B= T%*%Dc1/n

**Q** COmmenter les résultats ci-dessous. Quel vecteur propre associé à quelle valeur propre?

In [ ]:
A%*%B%*%gc-gc

In [ ]:
B%*%A%*%gr-gr

### 2.2 AFC et double ACP

**Q** Quelles sont les ACPs qui définissent l'AFC?

**Q** De quelles matrices faut-il alors calculer la SVD et par rapport à quelles métriques? 

**Attention**, la SVD est dite *généralisée* car calculée par rapport à des métriques spécifiques différentes de celle usuelle (matrice identité). Se reporter au [scénario précédent]() pour plus d'explications. Il n'est pas possible d'utiliser directement une fonction comme par exemple `prcomp` ou `princomp` pour calculer les ACPs conduisnat à l'AFC à cause de ce changement de métrique. Attention dans la construction des matrices, la matrice `T` est dans cet exemple carrée (autant de modalités lignes que colonnes) mais il ne faut pas confondre le rôle des métriques. 

In [ ]:
# SVD de 1/nT relative à Dr-1 et Dc-1
Drm12=diag(1/sqrt(gr));Dcm12=diag(1/sqrt(gc))
res=svd(Drm12%*%T%*%Dcm12/n)
res$d  

**Q** Pourquoi en AFC, centrer les matrices revient à éliminer la valeur propre 1 et le vecteur propre associé?

In [ ]:
L=res$d[-1]

In [ ]:
# Rechangement de métrique
U=diag(sqrt(gr))%*%res$u[,-1]
V=diag(sqrt(gc))%*%res$v[,-1]

In [ ]:
# Vérifications de la chi2-orthonormalité
t(U)%*%Dr1%*%U

In [ ]:
t(V)%*%Dc1%*%V

### 2.3 Calcul des composantes et graphiques

**Q** Quelles sont les composantes calculées ci-dessous?

In [ ]:
Cr=Dr1%*%U%*%diag(L)
Cc=Dc1%*%V%*%diag(L)

In [ ]:
# représentations graphiques
plot(Cc, type="n")
text(Cc, colnames(T))
abline(h=0,v=0)

**Q** Comment interpréter le graphique ci-dessus?

In [ ]:
plot(Cr, type="n")
text(Cr, rownames(T))
abline(h=0,v=0)

**Q** Comment interpréter le graphique ci-dessus?

## 3 Avec `FactoMineR`
Rien de bien compliqué dans ces instructions mais autant utiliser une librairie qui propose directement des graphiques de meilleure qualité avec une offre détaillée d'options.

In [ ]:
# Charger la librairie
library(FactoMineR)
# Analyse des correspondances
afc=CA(T, graph=F)
summary(afc)

Comparer les résultats avec ceux précédemment obtenus.

In [ ]:
plot(afc)

**Q** Comment se justifie la double représentation des modalités lignes et colonnes sur le même graphique?

**Q** Quelle interprétation en faire?